## Data Offers Key Players

In [2]:
import datetime as dt
import psycopg2
import pandas as pd
import plotly
from plotly import tools
import plotly.plotly as py
import plotly.graph_objs as go

from nbstyler import DATA_STYLE as s

plotly.offline.init_notebook_mode(connected=True) # run at the start of every ipython notebook to use plotly.offline

%matplotlib notebook
%matplotlib inline

In [9]:
conn = psycopg2.connect("dbname=jobsbg")
cur = conn.cursor()
cur.close()
datajobs_df = pd.read_sql_query('SELECT subm_date, subm_type, job_id, company_id, job_title, company_name FROM v_full_data_offers_history', conn, index_col='subm_date')
conn.close()

In [10]:
datajobs_df.index = pd.to_datetime(datajobs_df.index)


In [11]:
datajobs_df.head(2)

,subm_type,job_id,company_id,job_title,company_name
subm_date,,,,,
2017-09-27,submission,3994437,124912,Data Analyst,ПрайсуотърхаусКупърс Одит ООД
2017-09-27,submission,3994555,67058,ETL Developer,Adastra Bulgaria Ltd.


In [70]:
num_subm_df = datajobs_df.groupby('company_name')[['job_id']].count().sort_values(['job_id'], ascending=False)
num_subm_df.head(7)

,job_id
company_name,
bEYE Ltd,106
EXPERIAN BULGARIA EAD,64
Си Екс Джи ООД,62
Технементалс Технолоджис (България) ЕАД,42
ДОПАМИН ЕООД,36
КАЗИНО РЕШЕНИЯ ООД,34
Adecco Bulgaria Ltd.,31


In [12]:
# Uncomment the line below to export an HTML version of the chart.
# plotly.offline.plot(fig, filename = 'data_offers_key_players.html')

'file:///data/WORKSPACE/jpynb_Employment_Trends_Bulgaria/workbooks/data_offers_pie_and_bar.html'

In [11]:
from IPython.core.display import HTML
with open('../resources/styles/datum.css', 'r') as f:
    style = f.read()
HTML(style)